In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
import numpy as np
import os

In [4]:
gestures = ['raise', 'wave', 'other']

In [5]:
label_map = {label:num for num, label in enumerate(gestures)}
label_map

{'raise': 0, 'wave': 1, 'other': 2}

In [228]:
SEQUENCE_LENGTH = 30
DATASET_PATH = 'dataset'

In [230]:
sequences, labels = [], []
for action in gestures:
    for sequence in os.listdir(os.path.join(DATASET_PATH, action)):
        window = []
        for frame_num in range(SEQUENCE_LENGTH):
            res = np.load(os.path.join(DATASET_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [231]:
sequences = np.array(sequences)
labels = np.array(labels)

In [236]:
sequences[:,:,mask].shape

(240, 30, 258)

In [32]:
labels.shape

(180,)

In [248]:
X_train, X_test, y_train, y_test = train_test_split(sequences[:,:,mask], to_categorical(labels-1).astype(int), test_size=0.3)

In [256]:
X_train.shape, y_test.shape

((168, 30, 258), (72, 2))

In [115]:
y_train[:, 0].sum(), y_train[:, 1].sum(), y_train[:, 2].sum(), 

(43, 42, 41)

In [250]:
y_test[:, 0].sum(), y_test[:, 1].sum()#, y_test[:, 2].sum(), 

(24, 48)

In [219]:
X_train_new = X_train[:,:30, :]
y_train_new = y_train[:]
for i in [15,30]:
    X_train_new = np.concatenate([ X_train_new,  X_train[:,i:i+30, :]], 0)
    y_train_new = np.concatenate([y_train_new, y_train], 0)
X_train_new.shape, y_train_new.shape

((378, 30, 258), (378, 3))

In [218]:
X_test_new = X_test[:,:30, :]
y_test_new = y_test[:]
for i in [15,30]:
    X_test_new = np.concatenate([ X_test_new,  X_test[:,i:30+i, :]], 0)
    y_test_new = np.concatenate([y_test_new, y_test], 0)
X_test_new.shape, y_test_new.shape

((162, 30, 258), (162, 3))

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

In [39]:
log_dir = os.path.join('Logs_v2_more_data')
tb_callback = TensorBoard(log_dir=log_dir)

In [257]:
model = Sequential()
model.add(LSTM(60*2, return_sequences=True, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(60, return_sequences=False, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(len(gestures[1:]), activation='softmax'))

In [252]:
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_54 (LSTM)               (None, 30, 120)           181920    
_________________________________________________________________
lstm_55 (LSTM)               (None, 60)                43440     
_________________________________________________________________
dense_56 (Dense)             (None, 30)                1830      
_________________________________________________________________
dense_57 (Dense)             (None, 3)                 93        
Total params: 227,283
Trainable params: 227,283
Non-trainable params: 0
_________________________________________________________________


In [258]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [254]:
es_callback = EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=100, verbose=1,
    mode='auto', restore_best_weights=True
)

In [259]:
model.fit(X_train, y_train, epochs=1000, callbacks=[es_callback], validation_data=(X_test, y_test))

Epoch 1/1000
6/6 [==============================] - 2s 95ms/step - loss: 0.6934 - accuracy: 0.6012 - val_loss: 0.5817 - val_accuracy: 0.6667
Epoch 2/1000
6/6 [==============================] - 0s 42ms/step - loss: 0.5035 - accuracy: 0.7500 - val_loss: 0.5670 - val_accuracy: 0.7778
Epoch 3/1000
6/6 [==============================] - 0s 39ms/step - loss: 0.3995 - accuracy: 0.8214 - val_loss: 0.8171 - val_accuracy: 0.6528
Epoch 4/1000
6/6 [==============================] - 0s 36ms/step - loss: 0.4818 - accuracy: 0.7798 - val_loss: 0.3774 - val_accuracy: 0.8194
Epoch 5/1000
6/6 [==============================] - 0s 36ms/step - loss: 0.3546 - accuracy: 0.8393 - val_loss: 0.3293 - val_accuracy: 0.8333
Epoch 6/1000
6/6 [==============================] - 0s 36ms/step - loss: 0.4310 - accuracy: 0.8631 - val_loss: 0.5203 - val_accuracy: 0.7778
Epoch 7/1000
6/6 [==============================] - 0s 36ms/step - loss: 0.3463 - accuracy: 0.8690 - val_loss: 0.3457 - val_accuracy: 0.7917
Epoch 8/1000


Epoch 59/1000
6/6 [==============================] - 0s 39ms/step - loss: 0.0784 - accuracy: 0.9643 - val_loss: 0.3805 - val_accuracy: 0.8333
Epoch 60/1000
6/6 [==============================] - 0s 36ms/step - loss: 0.0584 - accuracy: 0.9881 - val_loss: 0.3916 - val_accuracy: 0.8194
Epoch 61/1000
6/6 [==============================] - 0s 36ms/step - loss: 0.0692 - accuracy: 0.9702 - val_loss: 0.3768 - val_accuracy: 0.8333
Epoch 62/1000
6/6 [==============================] - 0s 36ms/step - loss: 0.0742 - accuracy: 0.9821 - val_loss: 0.3939 - val_accuracy: 0.8889
Epoch 63/1000
6/6 [==============================] - 0s 37ms/step - loss: 0.0639 - accuracy: 0.9821 - val_loss: 0.4371 - val_accuracy: 0.7778
Epoch 64/1000
6/6 [==============================] - 0s 38ms/step - loss: 0.1261 - accuracy: 0.9464 - val_loss: 0.4565 - val_accuracy: 0.8194
Epoch 65/1000
6/6 [==============================] - 0s 38ms/step - loss: 0.0892 - accuracy: 0.9643 - val_loss: 0.5711 - val_accuracy: 0.8472
Epoch 

In [161]:
model2 = Sequential()
model2.add(LSTM(60*2, return_sequences=True, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model2.add(LSTM(60, return_sequences=False, activation='relu'))
model2.add(Dense(30, activation='relu'))
model2.add(Dense(len(gestures), activation='softmax'))
model2.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [146]:
(abs(X_train[:, :-1] - X_train[:, 1:])).shape

(126, 59, 258)

In [23]:
log_dir = os.path.join('Logs_v2_linear')
tb_callback = TensorBoard(log_dir=log_dir)

In [162]:
es_callback = EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=100, verbose=1,
    mode='auto', restore_best_weights=True
)
model2.fit(X_train, y_train, epochs=1000, callbacks=[es_callback], \
           validation_data=(X_test, y_test))

Epoch 1/1000
4/4 [==============================] - 2s 173ms/step - loss: 1.2374 - accuracy: 0.3651 - val_loss: 2.1493 - val_accuracy: 0.2778
Epoch 2/1000
4/4 [==============================] - 0s 83ms/step - loss: 8.5850 - accuracy: 0.3651 - val_loss: 1.1973 - val_accuracy: 0.3148
Epoch 3/1000
4/4 [==============================] - 0s 79ms/step - loss: 2.7639 - accuracy: 0.3810 - val_loss: 1.2232 - val_accuracy: 0.3333
Epoch 4/1000
4/4 [==============================] - 0s 79ms/step - loss: 1.0929 - accuracy: 0.3651 - val_loss: 1.0432 - val_accuracy: 0.3519
Epoch 5/1000
4/4 [==============================] - 0s 79ms/step - loss: 1.0687 - accuracy: 0.3651 - val_loss: 1.0730 - val_accuracy: 0.4074
Epoch 6/1000
4/4 [==============================] - 0s 79ms/step - loss: 1.0823 - accuracy: 0.3095 - val_loss: 1.0717 - val_accuracy: 0.5185
Epoch 7/1000
4/4 [==============================] - 0s 79ms/step - loss: 1.0785 - accuracy: 0.4444 - val_loss: 1.0605 - val_accuracy: 0.6481
Epoch 8/1000

Epoch 59/1000
4/4 [==============================] - 0s 82ms/step - loss: 0.7977 - accuracy: 0.6508 - val_loss: 1.1184 - val_accuracy: 0.5185
Epoch 60/1000
4/4 [==============================] - 0s 79ms/step - loss: 0.9861 - accuracy: 0.5397 - val_loss: 0.8680 - val_accuracy: 0.5926
Epoch 61/1000
4/4 [==============================] - 0s 81ms/step - loss: 0.8689 - accuracy: 0.5873 - val_loss: 0.8520 - val_accuracy: 0.6481
Epoch 62/1000
4/4 [==============================] - 0s 80ms/step - loss: 0.8695 - accuracy: 0.6508 - val_loss: 0.8533 - val_accuracy: 0.6296
Epoch 63/1000
4/4 [==============================] - 0s 78ms/step - loss: 0.8400 - accuracy: 0.6270 - val_loss: 0.7922 - val_accuracy: 0.6481
Epoch 64/1000
4/4 [==============================] - 0s 77ms/step - loss: 0.7803 - accuracy: 0.6429 - val_loss: 0.7531 - val_accuracy: 0.7407
Epoch 65/1000
4/4 [==============================] - 0s 86ms/step - loss: 0.7326 - accuracy: 0.6984 - val_loss: 0.7267 - val_accuracy: 0.6852
Epoch 

4/4 [==============================] - 0s 79ms/step - loss: 0.1688 - accuracy: 0.9127 - val_loss: 0.5462 - val_accuracy: 0.8519
Epoch 174/1000
4/4 [==============================] - 0s 78ms/step - loss: 0.1485 - accuracy: 0.9206 - val_loss: 0.4511 - val_accuracy: 0.9074
Epoch 175/1000
4/4 [==============================] - 0s 79ms/step - loss: 0.1255 - accuracy: 0.9524 - val_loss: 0.3926 - val_accuracy: 0.8704
Epoch 176/1000
4/4 [==============================] - 0s 79ms/step - loss: 0.1369 - accuracy: 0.9444 - val_loss: 0.6324 - val_accuracy: 0.8704
Epoch 177/1000
4/4 [==============================] - 0s 80ms/step - loss: 0.1333 - accuracy: 0.9444 - val_loss: 0.6005 - val_accuracy: 0.8519
Epoch 178/1000
4/4 [==============================] - 0s 77ms/step - loss: 0.1493 - accuracy: 0.9206 - val_loss: 0.8177 - val_accuracy: 0.7963
Epoch 179/1000
4/4 [==============================] - 0s 78ms/step - loss: 0.1967 - accuracy: 0.9206 - val_loss: 0.5052 - val_accuracy: 0.7963
Epoch 180/1000

4/4 [==============================] - 0s 80ms/step - loss: 0.0238 - accuracy: 0.9921 - val_loss: 0.9035 - val_accuracy: 0.7593
Epoch 288/1000
4/4 [==============================] - 0s 78ms/step - loss: 0.0402 - accuracy: 0.9841 - val_loss: 0.6652 - val_accuracy: 0.8519
Epoch 289/1000
4/4 [==============================] - 0s 79ms/step - loss: 0.0452 - accuracy: 0.9762 - val_loss: 0.6570 - val_accuracy: 0.8519
Epoch 290/1000
4/4 [==============================] - 0s 78ms/step - loss: 0.0247 - accuracy: 0.9921 - val_loss: 1.0761 - val_accuracy: 0.7778
Epoch 291/1000
4/4 [==============================] - 0s 86ms/step - loss: 0.0612 - accuracy: 0.9762 - val_loss: 0.7537 - val_accuracy: 0.8333
Epoch 292/1000
4/4 [==============================] - 0s 79ms/step - loss: 0.0663 - accuracy: 0.9762 - val_loss: 0.7871 - val_accuracy: 0.7778
Epoch 293/1000
4/4 [==============================] - 0s 86ms/step - loss: 0.0849 - accuracy: 0.9683 - val_loss: 0.8924 - val_accuracy: 0.7778
Epoch 294/1000

In [104]:
res = model.predict(X_test)

In [121]:
np.argmax(res, axis=1) , np.argmax(y_test, axis=1)

(array([1, 1, 2, 2, 0, 0, 1, 0, 1], dtype=int64),
 array([1, 0, 1, 0, 2, 1, 1, 0, 0, 0, 2, 0, 2, 2, 1, 2, 2, 2, 1, 2, 0, 1,
        2, 0, 2, 2, 0, 1, 0, 2, 2, 2, 1, 0, 0, 1, 0, 1, 2, 2, 1, 2, 0, 1,
        1, 2, 1, 1, 1, 0, 0, 1, 0, 2], dtype=int64))

In [263]:
model.save('gesture_wave_v2_without_dropout_30_old.h5')

In [93]:
model.load_weights('models/gesture_wave_v2.h5')

In [27]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [122]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1)#.tolist()
yhat = np.argmax(yhat, axis=1)#.tolist()

In [262]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[44,  4],
        [ 4, 20]],

       [[20,  4],
        [ 4, 44]]], dtype=int64)

In [124]:
accuracy_score(ytrue, yhat)

0.8148148148148148

In [261]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
accuracy_score(ytrue, yhat)

0.8888888888888888

In [102]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[113,   0],
        [  3,  55]],

       [[109,   6],
        [  0,  56]],

       [[114,   0],
        [  3,  54]]], dtype=int64)

In [164]:
yhat = model2.predict(X_train)
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
accuracy_score(ytrue, yhat)

0.9259259259259259